In [129]:
print('Mount my google drive')

from google.colab import drive
drive.mount('/content/drive')

Mount my google drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Rmat.csv

In [130]:
import pandas as pd
ratings_matrix = pd.read_csv('/content/drive/My Drive/Rmat.csv', sep=',', engine = 'python')


In [131]:
print(ratings_matrix.head())
print(ratings_matrix.loc['u1181'])
ratings_matrix.shape

        m1  m10  m100  m1000  m1002  m1003  m1004  m1005  m1006  m1007  ...  \
u1     5.0  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
u10    5.0  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
u100   NaN  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
u1000  5.0  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
u1001  4.0  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   

       m99  m990  m991  m992  m993  m994  m996  m997  m998  m999  
u1     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u10    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u100   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u1000  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u1001  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 3706 columns]
m1       3.0
m10      4.0
m100     NaN
m1000    NaN
m1002    NaN
        ... 
m994     4.0
m996   

(6040, 3706)

In [132]:
#import movie genres

all_movies = pd.read_csv('/content/drive/My Drive/movies.dat', sep='::', engine = 'python',
                     encoding="ISO-8859-1", header = None)
all_movies.columns = ['MovieID', 'Title', 'Genres']
# simplify movie genres: multiple genres to ‘Multiple’.
multiple_idx = pd.Series([("|" in movie) for movie in all_movies['Genres']])
all_movies.loc[multiple_idx, 'Genres'] = 'Multiple'

# Convert MovieID to string and prepend 'm'
all_movies['MovieID'] = 'm' + all_movies['MovieID'].astype(str)

# Set MovieID as the index
all_movies.set_index('MovieID', inplace=True)

print(all_movies.head(10))
print(all_movies.shape)

                                      Title    Genres
MovieID                                              
m1                         Toy Story (1995)  Multiple
m2                           Jumanji (1995)  Multiple
m3                  Grumpier Old Men (1995)  Multiple
m4                 Waiting to Exhale (1995)  Multiple
m5       Father of the Bride Part II (1995)    Comedy
m6                              Heat (1995)  Multiple
m7                           Sabrina (1995)  Multiple
m8                      Tom and Huck (1995)  Multiple
m9                      Sudden Death (1995)    Action
m10                        GoldenEye (1995)  Multiple
(3883, 2)


In [133]:
#take average of all the columns in the rating_matrix.. the most popular are the ones with the highest ratings exclude those with less than 10 ratings

# Dropping columns with less than 10 non-NA values
filtered_ratings = ratings_matrix.dropna(axis=1, thresh=10)

average_ratings = filtered_ratings.mean()
print(average_ratings)

m1       4.146846
m10      3.540541
m100     3.062500
m1000    3.050000
m1003    2.942149
           ...   
m994     4.095556
m996     2.906250
m997     3.357143
m998     3.010753
m999     3.283217
Length: 3260, dtype: float64


In [134]:
movies = all_movies.copy()

movies['Average_Rating'] = average_ratings

movies.dropna(inplace=True)

print(movies.head(100))

                                              Title    Genres  Average_Rating
MovieID                                                                      
m1                                 Toy Story (1995)  Multiple        4.146846
m2                                   Jumanji (1995)  Multiple        3.201141
m3                          Grumpier Old Men (1995)  Multiple        3.016736
m4                         Waiting to Exhale (1995)  Multiple        2.729412
m5               Father of the Bride Part II (1995)    Comedy        3.006757
...                                             ...       ...             ...
m106     Nobody Loves Me (Keiner liebt mich) (1994)  Multiple        3.583333
m107                  Muppet Treasure Island (1996)  Multiple        3.148438
m110                              Braveheart (1995)  Multiple        4.234957
m111                             Taxi Driver (1976)  Multiple        4.183871
m112                     Rumble in the Bronx (1995)  Multiple   

In [135]:
# Function to sort and get top 10
def get_top_10(group):
    return group.sort_values('Average_Rating', ascending=False).head(10)

# Group by 'Genres', apply the function
top_movies_by_genre = movies.groupby('Genres').apply(get_top_10).reset_index(level=0, drop=True)

# Display the result
print(top_movies_by_genre.head(20))

#save for future use

top_movies_by_genre.to_csv('top_movies_by_genre.csv')


                                              Title     Genres  Average_Rating
MovieID                                                                       
m2947                             Goldfinger (1964)     Action        3.971014
m2948                  From Russia with Love (1963)     Action        3.889754
m2949                                 Dr. No (1962)     Action        3.874613
m3384    Taking of Pelham One Two Three, The (1974)     Action        3.800725
m3310                               Kid, The (1921)     Action        3.790323
m2347         Pope of Greenwich Village, The (1984)     Action        3.736842
m2993                            Thunderball (1965)     Action        3.705446
m394                             Coldblooded (1995)     Action        3.666667
m3624                          Shanghai Noon (2000)     Action        3.654867
m3639           Man with the Golden Gun, The (1974)     Action        3.629067
m1254      Treasure of the Sierra Madre, The (1948) 

In [136]:
#System I - i know users favorite Genre what movies can I recommend

top_movies_by_genre = pd.read_csv('top_movies_by_genre.csv')

favorite_genre = 'Sci-Fi' #Multiple, Western, Sci-Fi, Action

print(top_movies_by_genre[top_movies_by_genre['Genres']==favorite_genre])

    MovieID                                 Title  Genres  Average_Rating
121   m1199                         Brazil (1985)  Sci-Fi        4.097481
122   m2010                     Metropolis (1926)  Sci-Fi        4.082474
123   m1206            Clockwork Orange, A (1971)  Sci-Fi        4.078112
124   m1301               Forbidden Planet (1956)  Sci-Fi        3.975871
125   m2660  Thing From Another World, The (1951)  Sci-Fi        3.896296
126   m3658         Quatermass and the Pit (1967)  Sci-Fi        3.652174
127   m3032                 Omega Man, The (1971)  Sci-Fi        3.540179
128   m2665   Earth Vs. the Flying Saucers (1956)  Sci-Fi        3.422535
129   m2661       It Came from Outer Space (1953)  Sci-Fi        3.361702
130    m680                     Alphaville (1965)  Sci-Fi        3.359375


In [137]:
import numpy as np


def normalize_ratings(matrix):
    # Calculate the mean for each row (user) excluding NaNs (missing ratings)
    row_means = matrix.mean(axis=1, skipna=True)

    # Subtract the mean from each element in the row
    norm_matrix = matrix.sub(row_means, axis=0)


    return norm_matrix

# Normalizing the ratings matrix
normalized_ratings = normalize_ratings(ratings_matrix)

print(normalized_ratings.head())
normalized_ratings.shape

             m1  m10  m100  m1000  m1002  m1003  m1004  m1005  m1006  m1007  \
u1     0.811321  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
u10    0.885287  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
u100        NaN  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
u1000  0.869048  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
u1001  0.347480  NaN   NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

       ...  m99  m990  m991  m992  m993  m994  m996  m997  m998  m999  
u1     ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u10    ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u100   ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u1000  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
u1001  ...  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 3706 columns]


(6040, 3706)

In [138]:
!pip install rpy2

In [139]:
%load_ext rpy2.ipython

In [140]:
%%R
install.packages("coop")


(as ‘lib’ is unspecified)







	‘/tmp/RtmpqsJEHV/downloaded_packages’



In [141]:


%reload_ext rpy2.ipython

# Passing the DataFrame to R
%R -i normalized_ratings


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [142]:
# Running R code
%%R
library(coop)
all_cosine_sim <- coop::cosine(as.matrix(normalized_ratings), use='pairwise.complete.obs')


In [143]:
#pass all_cosine_sim back to python

%R -o all_cosine_sim

In [144]:
import re

# Function to sort the columns and index by the numeric part of their names
def sort_dataframe_numerically(df):
    # Define a lambda to strip 'm' and convert to int for sorting
    extract_number = lambda x: int(x.lstrip('m'))

    # Sort the column names
    sorted_col_names = sorted(df.columns, key=extract_number)
    df = df.reindex(columns=sorted_col_names)

    # Sort the index if it contains 'm' prefixed numbers
    if df.index.dtype == 'object' and df.index.str.startswith('m').any():
        sorted_index = sorted(df.index, key=extract_number)
        df = df.reindex(sorted_index)

    return df

# Normalize the similarity score to be between 0 and 1
normalized_all_cosine_sim = (all_cosine_sim + 1) / 2

# Create a boolean matrix indicating where each user has rated each movie (ignoring NaNs)
user_movie_matrix = (~normalized_ratings.isna()).astype(int)

# Compute the number of common users for each movie pair
common_user_count = np.dot(user_movie_matrix.T, user_movie_matrix)

# Create a mask for movie pairs with at least three common users
valid_pairs_mask = common_user_count >= 3

# Apply the mask to the cosine similarity matrix
cosine_similarity_df = pd.DataFrame(np.where(valid_pairs_mask, normalized_all_cosine_sim, np.nan),
                                    index=normalized_ratings.columns, columns=normalized_ratings.columns)

# Set diagonal to NaN
np.fill_diagonal(cosine_similarity_df.values, np.nan)
cosine_similarity_df = sort_dataframe_numerically(cosine_similarity_df)
#save the full similarity matrix

cosine_similarity_df.to_csv('cosine_similarity_df.csv')

<ipython-input-144-3942643e0390>:20: RuntimeWarning: invalid value encountered in add
  normalized_all_cosine_sim = (all_cosine_sim + 1) / 2


In [145]:
import pandas as pd

# read in cosine_similarity_df.csv
cosine_similarity_df = pd.read_csv('cosine_similarity_df.csv', index_col=0)


def display_pairwise_similarity(S, movie_ids):
    # Extract the relevant rows and columns
    similarity_submatrix = S.loc[movie_ids, movie_ids]
    return pd.DataFrame(similarity_submatrix, index=movie_ids, columns=movie_ids).round(7)

# Assuming movie IDs are zero-indexed in your matrix
movie_ids = ['m1', 'm10', 'm100', 'm1510', 'm260', 'm3212']
pairwise_similarities = display_pairwise_similarity(cosine_similarity_df, movie_ids)
print(pairwise_similarities)


             m1       m10      m100  m1510      m260  m3212
m1          NaN  0.512105  0.392000    NaN  0.741148    NaN
m10    0.512105       NaN  0.547458    NaN  0.534334    NaN
m100   0.392000  0.547458       NaN    NaN  0.329694    NaN
m1510       NaN       NaN       NaN    NaN       NaN    NaN
m260   0.741148  0.534334  0.329694    NaN       NaN    NaN
m3212       NaN       NaN       NaN    NaN       NaN    NaN


In [146]:
import numpy as np


def refine_similarity_matrix_df(S_df):
    # Replace NaN with negative infinity for sorting within the DataFrame
    S_refined = S_df.fillna(-np.inf)
    # Get indices of top 30 values for each row
    top_30_indices = np.argsort(S_refined.values, axis=1)[:, -30:]

    # Create a new DataFrame to store the refined similarity matrix
    S_refined_top_30 = pd.DataFrame(np.nan, index=S_df.index, columns=S_df.columns)

    for i, indices in enumerate(top_30_indices):
        # Get the column names corresponding to the top 30 indices
        top_30_cols = S_df.columns[indices]
        # Assign the top 30 values to the new DataFrame
        S_refined_top_30.iloc[i][top_30_cols] = S_df.iloc[i][top_30_cols]


    return S_refined_top_30


# Refine the similarity matrix
S_refined = refine_similarity_matrix_df(cosine_similarity_df)
S_refined.head()



# Save the matrix to a CSV file
file_path = 'refined_similarity_matrix.csv'
pd.DataFrame(S_refined).to_csv(file_path,index=True)

In [148]:

def myIBCF(user, S_refined, top_movies_by_genre,all_movies):
    predictions = []
    for movie in user.index:
        if np.isnan(user[movie]):  # Movie is not rated by the user

            s_series = S_refined.loc[movie] #get the row of the similary matrix for the movie

            neighbors = s_series.dropna() #remove the na's

            numerator = sum(S_refined.loc[movie,i] * np.nan_to_num(user.loc[i]) for i in neighbors.index)

            user2 = user.copy()
            mask = (user2!=0) & (~np.isnan(user2))
            user2[mask]=1

            d = sum(S_refined.loc[movie,i] * np.nan_to_num(user2.loc[i]) for i in neighbors.index)
            # Only consider neighbors that have been rated by the user

            prediction = numerator / d if d != 0 else 0
            predictions.append((movie, prediction))



      # Handle case where fewer than 10 predictions are non-NA
    if len(predictions)<10:
      top = len(predictions)
    else:
      top = 10
    # Sort predictions in descending order and get the top top
    top_movies = sorted(predictions, key=lambda x: x[1], reverse=True)[:top]


  # Logic to handle fewer than 10 predictions
    # if fewer than 10 get the genre of the top movie and then get the most popular in that genra (system I)

    if top <10:
      if top == 0:
        #give them top Sci-Fi
        recommend = top_movies_by_genre[top_movies_by_genre['Genres']=='Sci-Fi']

      else:
        #remove the movies in the top_movies_by_genre already selected by user and by top_movies:
        top_movies_list = [i[0] for i in top_movies]
        movies_to_remove = top_movies_list + list(user2.index)
        movies_to_remove = list(set(movies_to_remove))
        bool_series = top_movies_by_genre.index.isin(movies_to_remove)
        filter_top_movies_by_genre = top_movies_by_genre.iloc[~bool_series]
        #get the genre of his top selecion
        top_genre = all_movies.loc[top_movies_list[0],'Genres']

        top_genre_df = top_movies_by_genre[top_movies_by_genre['Genres']==top_genre]

        add_on_movies = list(top_genre_df.iloc[:10-top]['MovieID'])
        tuple_movies = [(i,999) for i in add_on_movies ]
        top_movies = top_movies + tuple_movies





    # Logic for additional recommendations

    return top_movies

# read in cosine_similarity_df.csv
#S_refined = pd.read_csv('refined_similarity_matrix.csv', index_col=0)

# Extracting ratings for user 'u1181'

user_u1181_ratings = ratings_matrix.loc['u1181']
# Extracting ratings for user 'u1351'
user_u1351_ratings = ratings_matrix.loc['u1351']

blank_user = user_u1351_ratings.apply(lambda x: np.nan)

#A hypothetical user who rates movie “m1613” with 5 and movie “m1755” with 4

hypo_user = blank_user.copy()
hypo_user.name = 'hypo_user'
hypo_user['m1613'] =5
hypo_user['m1755'] =4


print( myIBCF(user_u1181_ratings,S_refined,top_movies_by_genre, all_movies ) )

[('m3732', 5.0), ('m749', 4.5265591633724265), ('m3899', 4.526066011687579), ('m1039', 4.0), ('m1235', 4.0), ('m1253', 4.0), ('m1734', 4.0), ('m1914', 4.0), ('m2082', 4.0), ('m2361', 4.0)]
